# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()


In [3]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
search_button = driver.find_element_by_id('checkbox')
search_button.click()



In [5]:
enter_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_button.click()


Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [6]:
license_button = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')

license_button.click()







### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [7]:
text_input = driver.find_element_by_id('txtTradeName')
text_input.send_keys('VAP%')

In [8]:
dropdown= Select(driver.find_element_by_xpath('//*[@id="slcJurisdiction"]'))
dropdown.select_by_visible_text('Statewide')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [9]:
submit_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit_button.click()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [11]:
field_title = driver.find_elements_by_class_name('searchfieldtitle')


for name in field_title:
    print(name.text)

1. VAPE IT STORE II
2. VAPE IT STORE I
3. VAPEPAD THE
4. VAPE FROG Pending *
5. VAPE FROG Pending *


'Location Address'

To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [12]:
detail_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[3]/a/img')
detail_button.click()

In [66]:
for button in field_title:
    detail_button.click()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


In [65]:
driver.find_elements_by_class_name()

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_x_path'

Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [13]:
def vape_scraper(row):
    driver.get(row['url'])
    
    name = driver.find_element_by_class_name('searchlistitem')
   
    
    return pd.Series({
        'name': name,
    
     })
         

In [14]:
field_title = driver.find_elements_by_class_name('searchfieldtitle')

field_title[0].find_element_by_tag_name('a').get_attribute('href')




'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bVlYUsxIIug%3D'

In [15]:
for links in field_title:
    try:
        print(links.find_element_by_tag_name('a').get_attribute('href'))
    except:
        print("no link available")

https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bVlYUsxIIug%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=aRZXzsEWLJI%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=0DrRbC%2Bbspk%3D
no link available
no link available


In [16]:
for name in field_title:
    try:
        print(name.text)
    except:
        print('no name listed')

1. VAPE IT STORE II
2. VAPE IT STORE I
3. VAPEPAD THE
4. VAPE FROG Pending *
5. VAPE FROG Pending *


In [17]:
vape_shops = []

for vape in field_title:
    
    row = {}
    
    name = vape.text
    
    row['name'] = name
    
    try:
        links = vape.find_element_by_tag_name('a').get_attribute('href')

        row['links'] = links
        
    except:
        print("no such link")

    vape_shops.append(row)

    

no such link
no such link


In [18]:
vape_shops

[{'name': '1. VAPE IT STORE II',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bVlYUsxIIug%3D'},
 {'name': '2. VAPE IT STORE I',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=aRZXzsEWLJI%3D'},
 {'name': '3. VAPEPAD THE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=0DrRbC%2Bbspk%3D'},
 {'name': '4. VAPE FROG Pending *'},
 {'name': '5. VAPE FROG Pending *'}]

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [143]:
Next_Button = driver.find_element_by_partial_link_text('Next')

Next_Button.click()

In [146]:
field_title = driver.find_elements_by_class_name('searchfieldtitle')

vape_shops = []


for vape in field_title:
    
    row = {}
    
    name = vape.text
    
    row['name'] = name
    
    try:
        links = vape.find_element_by_tag_name('a').get_attribute('href')

        row['links'] = links
        
    except:
        print("no such link")

    vape_shops.append(row)

Next_Button = driver.find_element_by_partial_link_text('Next')

Next_Button.click()


no such link
no such link


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [19]:
vape_shops = []


while True: 

    field_title = driver.find_elements_by_class_name('searchfieldtitle')

    
    for vape in field_title:
    
        row = {}
    
        name = vape.text
    
        row['name'] = name
    
        try:
            links = vape.find_element_by_tag_name('a').get_attribute('href')

            row['links'] = links
        
        except:
            print("no such link")

        vape_shops.append(row)

    Next_Button = driver.find_element_by_partial_link_text('Next')
    
    driver.execute_script("arguments[0].scrollIntoView(true)", Next_Button)

    Next_Button.click()

no such link
no such link
no such link
no such link
no such link
no such link
no such link
no such link
no such link
no such link


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


In [20]:
vape_shops

[{'name': '1. VAPE IT STORE II',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bVlYUsxIIug%3D'},
 {'name': '2. VAPE IT STORE I',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=aRZXzsEWLJI%3D'},
 {'name': '3. VAPEPAD THE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=0DrRbC%2Bbspk%3D'},
 {'name': '4. VAPE FROG Pending *'},
 {'name': '5. VAPE FROG Pending *'},
 {'name': '6. VAPE LOFT THE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ElkmonpAn5A%3D'},
 {'name': '7. VAPE N CIGAR',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=WNsXx1%2F7nrw%3D'},
 {'name': '8. VAPE DOJO',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=lV28HNF%2FLjs%3D'},
 {'name': '9. VAPE HAVEN Pending *'},
 {'name': '10. VAPORS LOUNGE Pending *'},
 {'name': '11. VAPORFI SOUTHSIDE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gHVCsxOq

In [153]:
vape_shops

[{'name': '1. VAPE IT STORE II',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1XQj3uqzxcE%3D'},
 {'name': '2. VAPE IT STORE I',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=cchN2tyBtEE%3D'},
 {'name': '3. VAPEPAD THE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=YiJVVPhXsUY%3D'},
 {'name': '4. VAPE FROG Pending *'},
 {'name': '5. VAPE FROG Pending *'},
 {'name': '6. VAPE LOFT THE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=O1Fomech%2Ftk%3D'},
 {'name': '7. VAPE N CIGAR',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gpBV%2FE%2BMoD4%3D'},
 {'name': '8. VAPE DOJO',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=v%2BwunPYSbtQ%3D'},
 {'name': '9. VAPE HAVEN Pending *'},
 {'name': '10. VAPORS LOUNGE Pending *'},
 {'name': '11. VAPORFI SOUTHSIDE',
  'links': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=MS3%2B

### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [21]:
vape_shops = []


while True: 

    field_title = driver.find_elements_by_class_name('searchfieldtitle')

    
    for vape in field_title:
    
        row = {}
    
        name = vape.text
    
        row['name'] = name
    
        try:
            links = vape.find_element_by_tag_name('a').get_attribute('href')

            row['url'] = links
        
        except:
            print("no such link")

        vape_shops.append(row)
    try:

        Next_Button = driver.find_element_by_partial_link_text('Next')
    
        driver.execute_script("arguments[0].scrollIntoView(true)", Next_Button)

        Next_Button.click()
    except:
        break

no such link
no such link
no such link
no such link
no such link
no such link
no such link
no such link
no such link
no such link


### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [23]:
df = pd.DataFrame(vape_shops)

In [24]:
df

,name,url
0,1. VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,2. VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,3. VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,4. VAPE FROG Pending *,NaN
4,5. VAPE FROG Pending *,NaN
5,6. VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,7. VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,8. VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,9. VAPE HAVEN Pending *,NaN
9,10. VAPORS LOUNGE Pending *,NaN


In [25]:
df.to_csv('vape-shops-basics.csv')

In [26]:
pd.read_csv('vape-shops-basics.csv')

,Unnamed: 0,name,url
0,0,1. VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,1,2. VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,2,3. VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,3,4. VAPE FROG Pending *,NaN
4,4,5. VAPE FROG Pending *,NaN
5,5,6. VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,6,7. VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,7,8. VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,8,9. VAPE HAVEN Pending *,NaN
9,9,10. VAPORS LOUNGE Pending *,NaN


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [103]:
pd.set_option('display.max_colwidth', -1)

df['url'][1]

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=aRZXzsEWLJI%3D'

### Use Selenium to visit that page

In [104]:
driver.get(df['url'][1])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [105]:
#Mailing Address
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text

'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804'

In [106]:
#Location Address

driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.split('\n')

['VAPE IT STORE I', '1724 N SALISBURY BLVD UNIT 2', 'SALISBURY, MD 21801']

In [119]:
name = address[:1]

name

['VAPE IT STORE II']

In [109]:
#License Information

driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split("\n")

['License Status: Issued',
 'License No.: 22375605',
 'Control No.: 22591855',
 'Date of Issue: 4/27/2018',
 'Months Paid: 12',
 'Exp. Date: 4/30/2019',
 'Subdivision: 22 Salisbury']

In [111]:
#Total Amount Paid: 


driver.find_elements_by_class_name('rt')[-1].text

'$ 24.00'

In [112]:
#Issued by:

driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text

'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'

In [113]:
driver.find_elements_by_tag_name('tbody')[5].text.split('\n')[1:]

['WICOMICO COUNTY, CLERK OF CIRCUIT COURT',
 'P.O. BOX 198',
 'SALISBURY, MARYLAND 21803-0198']

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [114]:
driver.get(df['url'][0])

vapeshop_data = []

row = {}


row['name'] =  driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.split('\n')[:-2]
row['location_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
row['mailing_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
row['license_information'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split("\n")
row['total_amount_paid'] = driver.find_elements_by_class_name('rt')[-1].text
row['issued_by'] = driver.find_elements_by_tag_name('tbody')[5].text.split('\n')[1:]

vapeshop_data.append(row)



In [115]:
vapeshop_data

[{'name': ['VAPE IT STORE II'],
  'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
  'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
  'license_information': ['License Status: Issued',
   'License No.: 22375606',
   'Control No.: 22884439',
   'Date of Issue: 4/27/2018',
   'Months Paid: 12',
   'Exp. Date: 4/30/2019',
   'Subdivision: 22 Salisbury'],
  'total_amount_paid': '$ 41.00',
  'issued_by': ['WICOMICO COUNTY, CLERK OF CIRCUIT COURT',
   'P.O. BOX 198',
   'SALISBURY, MARYLAND 21803-0198']}]

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [123]:
def vape_scraper(row):
    try:
        driver.get(row['url'])   
        
#         name = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.split('\n')[:1]
        location_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.replace('\n' , " ")
        mailing_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text.replace('\n' , " ")
        license_information = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split("\n")
        total_amount_paid = driver.find_elements_by_class_name('rt')[-1].text
        issued_by = driver.find_elements_by_tag_name('tbody')[5].text.split('\n')[1:]

    
    
        return pd.Series({
#                     'name' : name,
                    'location_address' : location_address,
                      'mailing_address' : mailing_address,
                      'license_information' : license_information,
                      'total_amount_paid' : total_amount_paid,
                      'issued_by' : issued_by
                     })
    except:
        return pd.Series({})
         

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [124]:
vape_total = df.apply(vape_scraper , axis=1)

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [125]:
vape_total


,issued_by,license_information,location_address,mailing_address,total_amount_paid
0,"[WICOMICO COUNTY, CLERK OF CIRCUIT COURT, P.O. BOX 198, SALISBURY, MARYLAND 21803-0198]","[License Status: Issued, License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE II 1015 S SALISBURY BLVD SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804",$ 41.00
1,"[WICOMICO COUNTY, CLERK OF CIRCUIT COURT, P.O. BOX 198, SALISBURY, MARYLAND 21803-0198]","[License Status: Issued, License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE I 1724 N SALISBURY BLVD UNIT 2 SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804",$ 24.00
2,"[ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT, 8 CHURCH CIRCLE, ROOM H-101, ANNAPOLIS, MARYLAND 21401]","[License Status: Issued, License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","VAPEPAD THE 2299 JOHNS HOPKINS ROAD GAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC 2504 ORCHARD KNOLL WAY ODENTON, MD 21113",$ 94.00
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,"[ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT, 8 CHURCH CIRCLE, ROOM H-101, ANNAPOLIS, MARYLAND 21401]","[License Status: Issued, License No.: 02310799, Control No.: 02686069, Date of Issue: 4/03/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","VAPE LOFT THE 185 MITCHELLS CHANCE RD EDGEWATER, MD 21037",DISBROW II EMERSON HARRINGTON,$ 154.00
6,"[WAYNE A. ROBEY, CLERK OF CIRCUIT COURT, 9250 BENDIX ROAD, COLUMBIA, MARYLAND 21045]","[License Status: Issued, License No.: 13343011, Control No.: 13856368, Date of Issue: 4/30/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 13 Howard County]","VAPE N CIGAR 7104 MINSTREL UNIT #7 COLUMBIA, MD 21045",DISCOUNT TOBACCO ESSEX LLC,$ 84.00
7,"[DONALD B. SEALING II, CLERK OF CIRCUIT COURT, 55 NORTH COURT STREET, WESTMINSTER, MARYLAND 21157-5155]","[License Status: Issued, License No.: 06327188, Control No.: 06946760, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 06 Westminster]","VAPE DOJO 330 ONE FORTY VILLAGE ROAD UNIT 15 WESTMINSTER, MD 21157",FAIRGROUND VILLAGE LLC,$ 179.00
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [126]:
vape_total.head()

,issued_by,license_information,location_address,mailing_address,total_amount_paid
0,"[WICOMICO COUNTY, CLERK OF CIRCUIT COURT, P.O. BOX 198, SALISBURY, MARYLAND 21803-0198]","[License Status: Issued, License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE II 1015 S SALISBURY BLVD SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804",$ 41.00
1,"[WICOMICO COUNTY, CLERK OF CIRCUIT COURT, P.O. BOX 198, SALISBURY, MARYLAND 21803-0198]","[License Status: Issued, License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE I 1724 N SALISBURY BLVD UNIT 2 SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804",$ 24.00
2,"[ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT, 8 CHURCH CIRCLE, ROOM H-101, ANNAPOLIS, MARYLAND 21401]","[License Status: Issued, License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","VAPEPAD THE 2299 JOHNS HOPKINS ROAD GAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC 2504 ORCHARD KNOLL WAY ODENTON, MD 21113",$ 94.00
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [127]:
vape_total.to_csv('vape-total.csv')

In [128]:
pd.read_csv('vape-total.csv')

,Unnamed: 0,issued_by,license_information,location_address,mailing_address,total_amount_paid
0,0,"['WICOMICO COUNTY, CLERK OF CIRCUIT COURT', 'P.O. BOX 198', 'SALISBURY, MARYLAND 21803-0198']","['License Status: Issued', 'License No.: 22375606', 'Control No.: 22884439', 'Date of Issue: 4/27/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 22 Salisbury']","VAPE IT STORE II 1015 S SALISBURY BLVD SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804",$ 41.00
1,1,"['WICOMICO COUNTY, CLERK OF CIRCUIT COURT', 'P.O. BOX 198', 'SALISBURY, MARYLAND 21803-0198']","['License Status: Issued', 'License No.: 22375605', 'Control No.: 22591855', 'Date of Issue: 4/27/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 22 Salisbury']","VAPE IT STORE I 1724 N SALISBURY BLVD UNIT 2 SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804",$ 24.00
2,2,"['ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT', '8 CHURCH CIRCLE, ROOM H-101', 'ANNAPOLIS, MARYLAND 21401']","['License Status: Issued', 'License No.: 02304705', 'Control No.: 02685930', 'Date of Issue: 4/05/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 02 Anne Arundel County']","VAPEPAD THE 2299 JOHNS HOPKINS ROAD GAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC 2504 ORCHARD KNOLL WAY ODENTON, MD 21113",$ 94.00
3,3,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN
5,5,"['ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT', '8 CHURCH CIRCLE, ROOM H-101', 'ANNAPOLIS, MARYLAND 21401']","['License Status: Issued', 'License No.: 02310799', 'Control No.: 02686069', 'Date of Issue: 4/03/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 02 Anne Arundel County']","VAPE LOFT THE 185 MITCHELLS CHANCE RD EDGEWATER, MD 21037",DISBROW II EMERSON HARRINGTON,$ 154.00
6,6,"['WAYNE A. ROBEY, CLERK OF CIRCUIT COURT', '9250 BENDIX ROAD', 'COLUMBIA, MARYLAND 21045']","['License Status: Issued', 'License No.: 13343011', 'Control No.: 13856368', 'Date of Issue: 4/30/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 13 Howard County']","VAPE N CIGAR 7104 MINSTREL UNIT #7 COLUMBIA, MD 21045",DISCOUNT TOBACCO ESSEX LLC,$ 84.00
7,7,"['DONALD B. SEALING II, CLERK OF CIRCUIT COURT', '55 NORTH COURT STREET', 'WESTMINSTER, MARYLAND 21157-5155']","['License Status: Issued', 'License No.: 06327188', 'Control No.: 06946760', 'Date of Issue: 4/05/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 06 Westminster']","VAPE DOJO 330 ONE FORTY VILLAGE ROAD UNIT 15 WESTMINSTER, MD 21157",FAIRGROUND VILLAGE LLC,$ 179.00
8,8,NaN,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN,NaN
